<a href="https://www.kaggle.com/code/sjagkoo7/predict-health-outcomes-of-horses-s3-ep22?scriptVersionId=144029426" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction
Predict whether or not a horse can survive based upon past medical conditions. There is an original dataset `horse-survival-dataset` - i have also used for reference.`Outcome` is target variable.This is a multi-class classification challenge to predict horse survival using the provided features. 

# Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualization like pie plot
import seaborn as sns # visualization like scatter plot 
pd.set_option('display.max_columns', 50) # display 50   columns by default
pd.set_option('display.max_rows', 50) # display 50 rows by default

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading the Dataset

In [ ]:
original=pd.read_csv('/kaggle/input/horse-survival-dataset/horse.csv')
train=pd.read_csv('/kaggle/input/playground-series-s3e22/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s3e22/test.csv')
submission=pd.read_csv('/kaggle/input/playground-series-s3e22/sample_submission.csv')

In [ ]:
# first three rows of original dataset
original.head(3)

In [ ]:
# first three rows of train dataset
train.head(3)

In [ ]:
# first three rows of test dataset
test.head(3)

In [ ]:
# first three rows of submission dataset
submission.head(3)

# Exploring the Dataset

In [ ]:
# Summary of Datasets
def summary(df):
    data=pd.DataFrame(index=df.columns)
    data['dtypes']=df.dtypes
    data['count']=df.count()
    data['#unique']=df.nunique()
    data['#missing']=df.isna().sum()
    data['missing%']=df.isna().sum()/len(df)*100
    data=pd.concat([data,df.describe().T.drop('count',axis=1)],axis=1)
    return data

In [ ]:
# Summary of original Dataset
summary(original).style.background_gradient(cmap='YlOrBr')

In [ ]:
# Summary of Training Dataset
summary(train).style.background_gradient(cmap='YlGnBu')

In [ ]:
# Summary of test Dataset
summary(test).style.background_gradient(cmap='viridis')

<div style = "color: White; display: fill;
              border-radius: 5px;
              background-color: #2C7FB8;
              font-size: 100%;
              font-family: Verdana">
    
## Insight:
* We have numerical, categorical and object columns
* Missing Values: The dataset contains a significant number of NA values, data imputation  will be required .
* The column hosptial number can be treated as  a categorical variable because it represents the different hospitals.
* Outcome: Target variable is  "outcome" variable. Possibilities include: lived, died, was euthanized.

In [ ]:
# original dataset - displaying rows if any have duplicate rows
original_duplicated_rows=original[original.duplicated()]
original_duplicated_rows

In [ ]:
# train dataset - displaying rows if any have duplicate rows
train_duplicated_rows=train[train.duplicated()]
train_duplicated_rows

In [ ]:
# test dataset - displaying rows if any have duplicate rows
test_duplicated_rows=test[test.duplicated()]
test_duplicated_rows

<div style = "color: White; display: fill;
              border-radius: 5px;
              background-color:  #2C7FB8;
              font-size: 100%;
              font-family: Verdana">
    
* There is no duplicated rows in original,train & test dataset

In [ ]:
#original dataset - displaying rows if any have null rows
original[original.isna().any(axis=1)]

In [ ]:
#train dataset - displaying rows if any have null rows
train[train.isna().any(axis=1)]

In [ ]:
#test dataset - displaying rows if any have null rows
test[test.isna().any(axis=1)]